In [1]:
import MySQLdb
import os
import pickle
import pandas as pd

In [2]:
conn = MySQLdb.connect(host = 'localhost', user = 'root', password = 'root')

In [3]:
cursor = conn.cursor()

In [ ]:
cursor.execute('create database EXPERIENCE')

In [4]:
cursor.execute('use EXPERIENCE')

0

In [5]:
cursor.execute('drop table login')
cursor.execute('create table LOGIN (username varchar(20), password varchar(20))')

0

In [6]:
cursor.execute('drop table newpost')
cursor.execute('create table newpost (username varchar(20), postid int NOT NULL AUTO_INCREMENT PRIMARY KEY, category varchar(15))')

0

In [7]:
cursor.execute('drop table category')
cursor.execute('create table category (username varchar(20), clickedid int, clickedcategory varchar(15))')

0

In [ ]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [162]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_comments(filename, idd):
    doc = load_doc(filename)
    
    if(len(doc)>0):
        i=1
        for line in doc.split('::'):
            tokens = line.split()
            if(len(tokens)>0):
                comment_id, comment_desc = tokens[0], tokens[1:]
                
                if (comment_id==str(idd)):
                    comment_desc = ' '.join(comment_desc)
                    print(str(i)+'. '+comment_desc.strip())
                    i=i+1    
    else:
        print("Post has no comments yet !")
    return

In [164]:
def comment(user):
    print("Enter postID to view post(enter 0 to exit) : ")
    idd = int(input())
    if(idd != 0):
        cursor.execute('select * from newpost ')
        result = cursor.fetchall()
        category='None'
        for i in result: 
            category = i[2]
            break
        cursor.execute('insert into category values(%s,%s,%s)',(user,idd,category))
        df = pd.read_csv('posts.csv')
        if(idd>len(df.index)):
            print("Post does not exist ! ")
        else:
            print("Title : "+df.iloc[idd-1][1])
            print()
            print("Description : "+df.iloc[idd-1][2])
            print()  
            print("Comments : ")
            print()
            filename = 'comments.txt'
            load_comments(filename, idd)
            while(1):
                print()
                print("1. Comment")
                print("2. Back")
                print("Enter choice : ")
                choice = int(input())
                if(choice==1):
                    print("Enter Comment : ")
                    comment = input()
                    comment = str(idd) +' '+ comment + '::'
                    file = open(filename, 'a')
                    file.write(comment)
                    file.close()
                    break
                elif(choice==2):
                    break
                else:
                    print("Incorrect input enter again !")

In [11]:
def seeposts(user):
    print("Recommended : ")
    recommended(user)
    print("New : ")
    new(user)
    while(1):
        print("1. View post")
        print("2. Back")
        print("Enter choice : ")
        choice = int(input())
        if(choice==1):
            comment(user)
        elif(choice == 2):
            break
        else:
            print("Incorrect input enter again !")

In [10]:
def newpost(user):
    while(1):   
        print("Enter title of problem : ")
        title = input()
        vectorizer = pickle.load(open("experiencevectorizerpickle.pickle", "rb"))
        t = vectorizer.transform([title])
        loaded_model = pickle.load(open('experience_classifier_model.sav', 'rb'))
        category = loaded_model.predict(t)[0]
        print("Enter description : ")
        description = input()
        print()
        print("Title : "+title)
        print("Description : "+description)

        print("1. Post")
        print("2. Rewrite")
        print("Enter choice : ")
        choice = int(input())
        if(choice == 1):
            break
        elif(choice == 2):
            continue
        else:
            print("Incorrect input")
    
    cursor.execute('insert into newpost(username, category) values (%s,%s)',(user, category))
    cursor.execute('SELECT * FROM newpost ORDER BY postid DESC LIMIT 1')
    results = cursor.fetchall()
    for i in results :
        continue
    raw = [[i[1],title,description]]
    df = pd.DataFrame(raw)
    if(i[1]==1):
        df.to_csv('posts.csv',index = False, mode = 'a')
    else:
        df.to_csv('posts.csv',index = False, mode = 'a', header = None)
    print()
    print("Posted !")

In [9]:
def login(user, password):
    while(1):
        print("1. New post")
        print("2. See posts")
        print("3. Your posts")
        print("4. Logout")
        print("Enter choice : ")
        choice = int(input())
        if(choice == 1):
            newpost(user)
        elif(choice == 2):
            seeposts(user)
        elif(choice == 3):
            yourposts(user)
        elif(choice == 4):
            print("Logged out !")
            break
        else:
            print("Incorrect choice, enter again !")

In [8]:
while(1):
    
    os.system('clear')
    print("1. Login")
    print("2. Sign up")
    print("3. Exit")
    print("Enter choice : ")
    choice = int(input())
    
    if(choice == 1):
        
        print("Enter username : ")
        user = input()
        print("Enter password : ")
        password = input()
        cursor.execute('select * from LOGIN')
        results = cursor.fetchall()
        k = 0
        
        for i in results:
            if(i[0]==user):
                k=k+1
                if(i[1]==password):
                    k=k+1
                    break
        
        if(k==2):
            print("successfully Logged in")
            login(user, password)
        else:
            print("Incorrect Username/ Password")
    
    elif(choice==2):
        
        while(1):
            print("Enter new username : ")
            user = input()
            cursor.execute('select * from LOGIN')
            result = cursor.fetchall()
            flag = 0
            for i in result:
                if (i[0]==user):
                    flag = 1
                    print("Username already taken. Enter again !")
            if(flag==0):
                break
            
        print("Enter new password : ")
        password = input()
        cursor.execute('insert into LOGIN values(%s,%s)',(user,password))
    
    elif(choice==3):
        print("Goodbye ! ")
        break
    
    else:
        print("Incorrect choice, enter again")

1. Login
2. Sign up
3. Exit
Enter choice : 
3
Goodbye ! 


In [20]:
newpost('rishabh')

Enter title of problem : 
2nd title
Enter description : 
2nd description

Title : 2nd title
Description : 2nd description
1. Post
2. Rewrite
Enter choice : 
1

Posted !


In [166]:
comment('rishabh')

Enter postID to view post(enter 0 to exit) : 
1
Title : 1st title

Description : 1st description

Comments : 

1. Hello mere bhai
2. Hello bhai sahab
3. Or kya haal hai

1. Comment
2. Back
Enter choice : 
2


In [92]:
cursor.execute('select * from category')
cursor.fetchall()

(('rishabh', 1, 'None'), ('rishabh', 1, 'None'), ('rishabh', 1, 'friends'))

In [19]:
df = pd.read_csv('posts.csv')
df.head(10)

,0,1,2
0,1,1st title,1st description


In [132]:
li = []
len(li)

0